In [1]:
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import LabelEncoder
import pickle

In [2]:
movies = pd.read_csv(r'C:\Users\paula\OneDrive\Curso\Henry\Proyecto Individual\Movies\movies_dataset.csv')
credits = pd.read_csv(r'C:\Users\paula\OneDrive\Curso\Henry\Proyecto Individual\Movies\credits.csv')

C:\Users\paula\AppData\Local\Temp\ipykernel_20456\724676636.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv(r'C:\Users\paula\OneDrive\Curso\Henry\Proyecto Individual\Movies\movies_dataset.csv')


In [3]:
movies[['revenue', 'budget']] = movies[['revenue', 'budget']].fillna(0)

In [4]:
# Asegurar que 'revenue' y 'budget' son numéricos y reemplazar nulos por 0
movies['revenue'] = pd.to_numeric(movies['revenue'], errors='coerce').fillna(0)
movies['budget'] = pd.to_numeric(movies['budget'], errors='coerce').fillna(0)

# Calcular 'return'
movies['return'] = np.where(movies['budget'] == 0, 0, movies['revenue'] / movies['budget'])

# Reemplazar divisiones por cero o valores nulos directamente
movies['return'] = movies['return'].replace([np.inf, np.nan], 0)


In [5]:
# Convertir release_date al formato datetime y manejar errores
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')

# Crear la columna release_year extrayendo el año
movies['release_year'] = movies['release_date'].dt.year

# Crear la columna release_month extrayendo el año
movies['release_month'] = movies['release_date'].dt.month

# Crear la columna release_dow extrayendo el año
movies['release_dow'] = movies['release_date'].dt.day_of_week

# Eliminar aquellos registros que quedaron nulos
movies = movies[movies['release_year'].notnull()]

In [6]:
# Asegurar que los valores de la columna sean tratados como diccionarios
movies['belongs_to_collection'] = movies['belongs_to_collection'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

# Extraer el nombre de la colección
movies['collection'] = movies['belongs_to_collection'].apply(
    lambda x: x['name'] if isinstance(x, dict) and 'name' in x else None
)

In [7]:
# Asegurar que los valores en la columna 'genres' sean listas de diccionarios
movies['genres'] = movies['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Extraer todos los nombres únicos de los géneros
all_genres = set()
for genre_list in movies['genres']:
    all_genres.update(genre['name'] for genre in genre_list)

# Crear el LabelEncoder y ajustarlo con los nombres únicos de los géneros
le = LabelEncoder()
le.fit(list(all_genres))

# Crear una nueva columna con los géneros codificados
movies['genres_encoded'] = movies['genres'].apply(
    lambda genre_list: le.transform([genre['name'] for genre in genre_list]).tolist()
)

# Guardar el LabelEncoder
with open('label_encoder.pkl', 'wb') as file:
    pickle.dump(le, file)


In [8]:
movies = movies.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage', 'spoken_languages', 'production_countries', 'belongs_to_collection', 'production_companies', 'original_language','tagline', 'genres', 'overview', 'runtime', 'status'], errors='ignore')


In [9]:
movies['popularity'] = pd.to_numeric(movies['popularity'], errors='coerce')


In [10]:
# Guardar el dataset transformado
movies.to_csv(r'Datasets/transformed_movies.csv', index=False)

In [11]:
# Convertir la columna `crew` en listas de diccionarios
credits['crew'] = credits['crew'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Extraer el nombre de los directores
credits['director_name'] = credits['crew'].apply(
    lambda x: next((person['name'] for person in x if person.get('job') == 'Director'), None)
)

# Asegurarse de que la columna `cast` se convierta en listas de diccionarios
credits['cast'] = credits['cast'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Extraer los nombres de todos los actores
credits['actors'] = credits['cast'].apply(
    lambda x: [actor['name'] for actor in x] if isinstance(x, list) else []
)

# Crear un nuevo DataFrame reducido
reduced_credits = credits[['id', 'director_name', 'actors']]

# Guardar el dataset reducido
reduced_credits.to_csv(r'Datasets/reduced_credits.csv', index=False)